# 2D DC Resistivity Sensitivity analysis

```{admonition} Intermediate notebook
:class: caution
This tutorial focusses on intermediate level functionality within SimPEG. Basic functionality within SimPEG is not discussed in detail, as we assume the user is already familiar.
```

```{admonition} Light-weight notebook
:class: hint
This tutorial requires minimal computational resources and can be executed quickly in the background while other computer processes are running.
```

**Author:** Williams Lima

</br>

**Keywords:** DC Resistivity inversion, sensitivity matrix.

</br>

**Summary:**
The sensitivity matrix is a very important tool for the design of surveys and the implementation of inversion algorithms in geophysics. It can give us qualitative and, to some extent, quantitative information about the sensitivity of the data to perturbations in the physical properties (e.g electrical conductivities) underground. In turn, it can help us design optimized surveys for specific targets. Moreover, it is also of great value in a later interpretation stage, as a tool to assess the resolvability of particular features obtained by the use of inversion methods. In this tutorial we'll use SimPEG functionalities to look at the sensitivity matrix for the DC resistivity inverse problem. We will use a number of different techniques to analyze the sensitivity patterns for a few commonly used electrode arrays. As study cases, we will apply these techniques for both a homogenous half space and to a real data set in the context of a mineral exploration project.

Although we use the DC resistivity problem as a leitmotif in this tutorial, the techniques presented here are general enough and can be applied to many other geophysical methods.

Enjoy!
</br>

**Learning Objectives:**

- How to get the sensitivity matrix for a DC resistivity simulation by using utility functions in SimPEG.
- Exploring the different sensitivity patterns for a number of common electrode arrays.
- How to plot information contained in the sensitivity matrix.
- Look at the sensitivity matrix for a real data set.

</br>

## Import modules

Here, we import all of the functionality required to run the notebook for the tutorial exercise. All of the functionality specific to DC resistivity is imported from [SimPEG.electromagnetics.static.resistivity](myst:SimPEG#SimPEG.electromagnetics.static.resistivity).

We also import some useful utility functions from [SimPEG.utils](myst:SimPEG#SimPEG.utils). To simulate DC resistivity data, we need to define our problem geometry on a numerical grid (or mesh).
To generate the mesh, we used the [discretize](https://discretize.simpeg.xyz/en/main) package.

In [ ]:
# SimPEG functionality
from SimPEG.electromagnetics.static import resistivity as dc
from SimPEG import maps

# discretize functionality
from discretize import TreeMesh, TensorMesh
from discretize.utils import refine_tree_xyz

# Common Python functionality
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

# SimPEG inversion machinery
from SimPEG import (
    Data, maps,
    data_misfit, regularization, optimization, inverse_problem, 
    inversion, directives
) 

from SimPEG.electromagnetics.static import utils as dc_utils

# linear solvers
try: 
    from pymatsolver import Pardiso as Solver  # this is a fast linear solver 
except ImportError:
    from SimPEG import SolverLU as Solver  # this will be slower

## Homogeneous half space

### Define surveys
There are many different possible combinations of current and potential electrodes locations that could be used to create a DC resistivity survey. Maybe some of those combinations would be better than others for a particular case under specific constraints such as cost, physical constraints, time and sensitivity to a particular target at depth. The sensitivity analysis, that we are going to perform in this tutorial, will help us with the task of assessing the performance of a given electrode array with respect to its sensitivity to conductivity anomalies around a particular target region in the subsurface. For more information about DC resistivity modeling in SimPEG check out the tutorial [2.5D DC Resistivity Simulation](05-dcr/fwd_dcr_2d).

In the figure below, we can see a few of the most common array configurations used in the industry and academic works.

```{figure}https://gpg.geosci.xyz/_images/figure6.gif
:width: 350px
:align: center

DC resistivity arrays ([image source](https://gpg.geosci.xyz/_images/figure6.gif))
```

**Two simple surveys in a homogeneous medium**

Our first step is to create two simple surveys, Wenner alpha and Wenner beta. Each of theses surveys will contain one pair of current electrodes and one pair of potential electrodes only. The extension to any number of current/potential electrodes is straightforward and will be left as an exercise.



In [ ]:
# Define survey line parameters for Wenner surveys
data_type = "volt"
src_list = []

# Wenner alpha
C_wenner_alpha = np.array([[-15.0, 0.0], [15.0, 0.0]])
P_wenner_alpha = np.array([[-5.0, 0.0], [5.0, 0.0]])

rx_wenner_alpha = dc.receivers.Dipole(locations_m=P_wenner_alpha[0], locations_n=P_wenner_alpha[1], data_type=data_type) 
rx_list_wenner_alpha = [rx_wenner_alpha]

src_list.append(dc.sources.Dipole(location_a=C_wenner_alpha[0], location_b=C_wenner_alpha[1], receiver_list=rx_list_wenner_alpha))

# Dipole-Dipole
C_dipole_dipole = np.array([[-15.0, 0.0], [-5.0, 0.0]])
P_dipole_dipole = np.array([[5.0, 0.0], [15.0, 0.0]])

rx_dipole_dipole = dc.receivers.Dipole(locations_m=P_dipole_dipole[0], locations_n=P_dipole_dipole[1], data_type=data_type) 
rx_list_dipole_dipole = [rx_dipole_dipole]

src_list.append(dc.sources.Dipole(location_a=C_dipole_dipole[0], location_b=C_dipole_dipole[1], receiver_list=rx_list_dipole_dipole))

survey = dc.Survey(source_list=src_list)

### Create mesh

In [ ]:
dh = 0.25  # base cell width
dom_width_x = 150.0  # domain width x
dom_width_z = 100.0  # domain width z
nbcx = 2 ** int(np.round(np.log(dom_width_x / dh) / np.log(2.0)))  # num. base cells x
nbcz = 2 ** int(np.round(np.log(dom_width_z / dh) / np.log(2.0)))  # num. base cells z

# Define the base mesh with top at z = 0 m.
hx = [(dh, nbcx)]
hz = [(dh, nbcz)]
mesh = TreeMesh([hx, hz], x0="CN")

# Refine core mesh region
mesh.refine_box([-50,-50],[50,0], levels=-1, finalize=False)

mesh.finalize()

### Define the conductivity model

In [ ]:
background_resistivity = 50 # Ohm.m
background_conductivity = 1/background_resistivity
nC = mesh.nC
conductivity_model = background_conductivity * np.ones(nC)

conductivity_map = maps.IdentityMap()

In [ ]:
# DC simulation for a conductivity model
simulation_con = dc.simulation_2d.Simulation2DNodal(
    mesh, survey=survey, sigmaMap=conductivity_map
)

### Plot sensitivity

**Get the sensitivity matrix**

In [ ]:
J = simulation_con.getJ(conductivity_model).copy()
J_wenner_alpha = J[0,:]/np.max(abs(J[0,:]))
J_dipole_dipole = J[1,:]/np.max(abs(J[1,:]))

**Make plots**

In [ ]:
##
#
fig, axs = plt.subplots(1,2, figsize=(10,2))
plt.jet()

norm = mpl.colors.SymLogNorm(
    linthresh=0.005, linscale=1e-4,
    vmin=-1.0, vmax=1.0, base=10)

Js = np.c_[J_wenner_alpha, J_dipole_dipole] 
C_locs = np.r_[C_wenner_alpha, C_dipole_dipole]
P_locs = np.r_[P_wenner_alpha, P_dipole_dipole]

plt_titles = ["Wenner alpha", "dipole-dipole"]
i = 0
for ax in axs:
    mesh.plot_image(
        Js[:,i],
        ax=ax,
        grid=False,
        pcolor_opts={"norm": norm}
    )
    
    xmin = -50; xmax = 50
    zmin = -20; zmax = 0
    ax.set_xlim(xmin, xmax)
    ax.set_ylim(zmin, zmax)
    ax.set_title(plt_titles[i], loc="left")
    ax.set_xlabel("x (m)")
    ax.set_ylabel("z (m)")

    xc = (C_locs[2*i,0] - xmin)/(xmax - xmin)
    ax.annotate('', xy=(xc, 1), xycoords='axes fraction', xytext=(xc, 1.1), 
            arrowprops=dict(arrowstyle="->", color='r'))
    xc = (C_locs[2*i+1,0] - xmin)/(xmax - xmin)
    ax.annotate('', xy=(xc, 1), xycoords='axes fraction', xytext=(xc, 1.1), 
            arrowprops=dict(arrowstyle="->", color='r'))
    xc = (P_locs[2*i,0] - xmin)/(xmax - xmin)
    ax.annotate('', xy=(xc, 1), xycoords='axes fraction', xytext=(xc, 1.075), 
            arrowprops=dict(arrowstyle="->", color='b'))
    xc = (P_locs[2*i+1,0] - xmin)/(xmax - xmin)
    ax.annotate('', xy=(xc, 1), xycoords='axes fraction', xytext=(xc, 1.075), 
            arrowprops=dict(arrowstyle="->", color='b'))

    i = i + 1

ax2 = fig.add_axes([0.25, -0.3, 0.5, 0.1])
cbar = mpl.colorbar.ColorbarBase(
    ax2, norm=norm, orientation="horizontal"
)
cbar.set_label("Sensitivity", rotation=0, labelpad=15, size=12)

***Cumulative sensitivity of a given array***

Let's experiment with another way to look at the sensitivity matrix. This time we are going to plot the **cumulative sensitivity** $CS$ defined as
$$
 CS = \frac{\sum_{i=1}^P S_i}{\sum_{i=1}^M S_i}
$$
where $S$ contains a particular row of $\left|J\right|$ sorted in decreasing order.

We took inspiration for this plot from {cite}` https://doi.org/10.2136/vzj2003.4160`.


In [ ]:
# Define survey line parameters
survey_type = "dipole-dipole"
dimension_type = "2D"
data_type = "volt"

src_list = []
C_locs = []
P_locs = []

# Array 1
C = np.array([[-15.0, 0.0], [-5.0, 0.0]])
P = np.array([[5.0, 0.0], [15.0, 0.0]])

C_locs.append(C)
P_locs.append(P)

rx = dc.receivers.Dipole(locations_m=P[0], locations_n=P[1], data_type=data_type) 
rx_list = [rx]

src_list.append(dc.sources.Dipole(location_a=C[0], location_b=C[1], receiver_list=rx_list))

# Array 2
C = np.array([[-30.0, 0.0], [-20.0, 0.0]])
P = np.array([[20.0, 0.0], [30.0, 0.0]])

C_locs.append(C)
P_locs.append(P)

rx = dc.receivers.Dipole(locations_m=P[0], locations_n=P[1], data_type=data_type) 
rx_list = [rx]

src_list.append(dc.sources.Dipole(location_a=C[0], location_b=C[1], receiver_list=rx_list))

# Define survey
survey = dc.Survey(src_list)

In [ ]:
# DC simulation for a conductivity model
simulation_con = dc.simulation_2d.Simulation2DNodal(
    mesh, survey=survey, sigmaMap=conductivity_map
)

In [ ]:
J = simulation_con.getJ(conductivity_model).copy()

In [ ]:
##
# Interpolate to a regular grid for creating the plots
xmin = -50; xmax = 50.0
grid_nx = 500
zmin = -20.0; zmax = 0.0
grid_nz = 200
[xgrid, zgrid] = np.meshgrid(np.linspace(xmin, xmax, grid_nx), np.linspace(zmin, zmax, grid_nz))
locs = np.vstack([xgrid.ravel(), zgrid.ravel()])
Q = mesh.get_interpolation_matrix(locs.T)


In [ ]:
fig, axs = plt.subplots(1,2, figsize=(10,2))
levels = [0.95, 0.7, 0.5, 0.2]

i = 0
for ax in axs:
    Js = (Q@J[i,:].T)
    ##
    # Fill cells with corresponding cumulative sensitivity values
    Z = np.ones(grid_nz*grid_nx)
    
    N = np.size(Js)
    CS_all = np.sum(abs(Js))
    Js_sort_idx = (-abs(Js)).argsort()[:N]
    Js_sort = abs(Js[Js_sort_idx])
    for perc in levels:
        CS = 0
        for j in range(N):
            CS = CS + abs(Js_sort[j])/CS_all        
            if CS >= perc:
                break
        Z[Js_sort_idx[0:j+1]] = perc

    Z = Z.reshape((grid_nz, grid_nx))
    
    ##
    # Plot
    ax.contourf(xgrid, zgrid, Z, levels=levels.sort(), cmap='GnBu_r')
    ccs = ax.contour(xgrid, zgrid, Z,
                     levels=levels.sort(),
                      colors='w', linewidths=0.75)
    ax.clabel(ccs, ccs.levels, inline=False, 
              fmt='%2.1f', fontsize=10, colors="k")

    ##
    # Plot electrodes locations
    xc = (C_locs[i][0,0] - xmin)/(xmax - xmin)
    ax.annotate('', xy=(xc, 1), xycoords='axes fraction', xytext=(xc, 1.1), 
            arrowprops=dict(arrowstyle="->", color='r'))
    xc = (C_locs[i][1,0] - xmin)/(xmax - xmin)
    ax.annotate('', xy=(xc, 1), xycoords='axes fraction', xytext=(xc, 1.1), 
            arrowprops=dict(arrowstyle="->", color='r'))
    xc = (P_locs[i][0,0] - xmin)/(xmax - xmin)
    ax.annotate('', xy=(xc, 1), xycoords='axes fraction', xytext=(xc, 1.075), 
            arrowprops=dict(arrowstyle="->", color='b'))
    xc = (P_locs[i][1,0] - xmin)/(xmax - xmin)
    ax.annotate('', xy=(xc, 1), xycoords='axes fraction', xytext=(xc, 1.075), 
            arrowprops=dict(arrowstyle="->", color='b'))

    i = i + 1
    
ax2 = fig.add_axes([0.25, -0.3, 0.5, 0.1])
cbar = mpl.colorbar.ColorbarBase(ax2, orientation="horizontal", cmap='GnBu_r')
cbar.set_label("Cumulative normalized sensitivity", rotation=0, labelpad=15, size=12)

## Real data example: The Century Deposit DC resistivity inversion

In this section we perform a sensitivity analysis for the solution obtained in the inversion of a real data set. We will build upon the tutorial that can be found at [transform2020](https://github.com/simpeg/transform-2020-simpeg) created by Lindsey Heagy. You are also welcome to watch her associated presentation on youtube: [Tutorial: Geophysical Inversion in SimPEG](https://www.youtube.com/watch?v=jZ7Sj9cnnso&t=3250s)

### Initial setup

We will perform the following steps:
- Read information about the survey from a data file on disk
- Create a python dictionary containing:
  - a_locations: the positive source electrode locations (numpy array) 
  - b_locations: the negative source electrode locations (numpy array) 
  - m_locations: the receiver locations (list of numpy arrays)
  - n_locations: the receiver locations (list of numpy arrays)
  - observed_data: observed data (list of numpy arrays)
  - standard_deviations: assigned standard deviations (list of numpy arrays)
  - n_sources: number of sources (int)

In [ ]:
line = "46800E"
dc_data_file = f"./century/{line}/{line[:-1]}POT.OBS"

def read_dcip_data(filename, verbose=False):
    """
    Read in a .OBS file from the Century data set into a python dictionary. 
    The format is the old UBC-GIF DCIP format.
    
    Parameters
    ----------
    filename : str
        Path to the file to be parsed
    
    verbose: bool
        Print some things? 
    
    
    Returns
    -------
    dict
        A dictionary with the locations of
        - a_locations: the positive source electrode locations (numpy array) 
        - b_locations: the negative source electrode locations (numpy array) 
        - m_locations: the receiver locations (list of numpy arrays)
        - n_locations: the receiver locations (list of numpy arrays)
        - observed_data: observed data (list of numpy arrays)
        - standard_deviations: assigned standard deviations (list of numpy arrays)
        - n_sources: number of sources (int)
    
    """
    
    # read in the text file as a numpy array of strings (each row is an entry)
    contents = np.genfromtxt(filename, delimiter=' \n', dtype='U')
    
    # the second line has the number of sources, current, and data type (voltages if 1)
    n_sources = int(contents[1].split()[0])
    
    if verbose is True: 
        print(f"number of sources: {n_sources}")
    
    # initialize storage for the electrode locations and data
    a_locations = np.zeros(n_sources)
    b_locations = np.zeros(n_sources)
    m_locations = []
    n_locations = []
    observed_data = []
    standard_deviations = []
    
    # index to track where we have read in content 
    content_index = 1 
    
    # loop over sources 
    for i in range(n_sources):
        # start by reading in the source info 
        content_index = content_index + 1  # read the next line
        a_location, b_location, nrx = contents[content_index].split()  # this is a string

        # convert the strings to a float for locations and an int for the number of receivers
        a_locations[i] = float(a_location)
        b_locations[i] = float(b_location)
        nrx = int(nrx)

        if verbose is True: 
            print(f"Source {i}: A-loc: {a_location}, B-loc: {b_location}, N receivers: {nrx}")

        # initialize space for receiver locations, observed data associated with this source
        m_locations_i, n_locations_i = np.zeros(nrx), np.zeros(nrx)
        observed_data_i, standard_deviations_i = np.zeros(nrx), np.zeros(nrx)

        # read in the receiver info 
        for j in range(nrx):
            content_index = content_index + 1  # read the next line
            m_location, n_location, datum, std = contents[content_index].split()

            # convert the locations and data to floats, and store them
            m_locations_i[j] = float(m_location)
            n_locations_i[j] = float(n_location)
            observed_data_i[j] = float(datum)
            standard_deviations_i[j] = float(std)

        # append the receiver info to the lists
        m_locations.append(m_locations_i)
        n_locations.append(n_locations_i)
        observed_data.append(observed_data_i)
        standard_deviations.append(standard_deviations_i)
    
    return {
        "a_locations": a_locations,
        "b_locations": b_locations, 
        "m_locations": m_locations,
        "n_locations": n_locations,
        "observed_data": observed_data, 
        "standard_deviations": standard_deviations,
        "n_sources": n_sources, 
    }

dc_data_dict = read_dcip_data(dc_data_file)

### Create survey

Here we create a SimPEG Survey object representing the experiment layout.

In [ ]:
# initialize an empty list for each 
source_list = []

for i in range(dc_data_dict["n_sources"]):
    
    # receiver electrode locations in 2D 
    m_locs = np.vstack([
        dc_data_dict["m_locations"][i], 
        np.zeros_like(dc_data_dict["m_locations"][i])
    ]).T
    n_locs = np.vstack([
        dc_data_dict["n_locations"][i],
        np.zeros_like(dc_data_dict["n_locations"][i])
    ]).T
    
    # construct the receiver object 
    receivers = dc.receivers.Dipole(locations_m=m_locs, locations_n=n_locs)
    
    # construct the source 
    source = dc.sources.Dipole(
        location_a=np.r_[dc_data_dict["a_locations"][i], 0.],
        location_b=np.r_[dc_data_dict["b_locations"][i], 0.],
        receiver_list=[receivers]
    )
    
    # append the new source to the source list
    source_list.append(source)

survey = dc.Survey(source_list=source_list)

dc_data = Data(
    survey=survey, 
    dobs=np.hstack(dc_data_dict["observed_data"]),
    standard_deviation=np.hstack(dc_data_dict["standard_deviations"])
)

### Forward simulation components

***Mesh design***

We will create a [discretize](https://discretize.simpeg.xyz/en/main) TensorMesh.

In [ ]:
min_electrode_spacing = np.min(np.abs(survey.locations_a[:, 0] - survey.locations_b[:, 0]))
n_cells_per_spacing = 4

dx = min_electrode_spacing / n_cells_per_spacing
dz = min_electrode_spacing / n_cells_per_spacing

electrode_locations = np.c_[
    survey.locations_a,
    survey.locations_b,
    survey.locations_m,
    survey.locations_n,
]

unique_locations = np.unique(
    np.reshape(electrode_locations, (4 * survey.nD, 2)), axis=0
)

core_domain_x = np.r_[
    unique_locations[:,0].min(),
    unique_locations[:,0].max()
]

# find the maximum spacing between source, receiver midpoints
mid_ab = (survey.locations_a + survey.locations_b)/2
mid_mn = (survey.locations_m + survey.locations_n)/2
separation_ab_mn = np.abs(mid_ab - mid_mn)
max_separation = separation_ab_mn.max()

core_domain_z = np.r_[-max_separation/3., 0.]

n_core_extra = 4 
n_core_x = np.ceil(np.diff(core_domain_x)/dx) + n_core_extra*2  # on each side
n_core_z = np.ceil(np.diff(core_domain_z)/dz) + n_core_extra  # just below

padding_factor = 1.3
n_pad_x = 10
n_pad_z = 10

hx = [(dx, n_pad_x, -padding_factor), (dx, n_core_x), (dx, n_pad_x, padding_factor)]
hz = [(dz, n_pad_z, -padding_factor), (dz, n_core_z)]

mesh = TensorMesh([hx, hz])

# origin of the mesh
mesh_hx = mesh.h[0]
mesh_hy = mesh.h[1]
mesh.x0 = np.r_[
    -mesh_hx[:n_pad_x + n_core_extra].sum() + core_domain_x.min(),
    -mesh_hy.sum()
]

***SimPEG forward simulation objects creation***

The SimPEG simulation object for this particular problem is an object of type [SimPEG.electromagnetics.static.resistivity.Simulation2DNodal](myst:SimPEG#SimPEG.electromagnetics.static.resistivity.Simulation2DNodal). We will also work with the natual logarithm of the conductivity.

In [ ]:
# forward simulation 
# Use Exponential Map: m = log(rho)
mapping = maps.ExpMap(mesh)

# Generate 2.5D DC problem
simulation_dc = dc.Simulation2DNodal(
    mesh, rhoMap=mapping, solver=Solver, survey=survey, storeJ=True
)

### Inversion

***Inversion: Data misfit***

In [ ]:
dmisfit = data_misfit.L2DataMisfit(data=dc_data, simulation=simulation_dc)

***Inversion: Regularization***

In [ ]:
mesh_hx = mesh.h[0]
reg = regularization.WeightedLeastSquares(
    mesh, 
    alpha_s=1./mesh_hx.min()**2,  # we will talk about these choices in notebook 3 on inversion
    alpha_x=1, 
    alpha_y=1,  # since this is a 2D problem, the second dimension is "y" 
)

***Inversion: Final touches***

In [ ]:
opt = optimization.InexactGaussNewton(maxIter=20, maxIterCG=20)
inv_prob = inverse_problem.BaseInvProblem(dmisfit, reg, opt)

# directives
beta_est = directives.BetaEstimate_ByEig(beta0_ratio=1)

#coolFac = 4
coolFac = 1
beta_schedule = directives.BetaSchedule(coolingFactor=coolFac, coolingRate=2)
target = directives.TargetMisfit()

inv = inversion.BaseInversion(
    inv_prob, directiveList=[beta_est, beta_schedule, target]
)


***Inversion: (Finally) run the inversion***

In [ ]:
apparent_resistivity = dc_utils.apparent_resistivity_from_voltage(dc_data.survey, dc_data.dobs)
rho0 = np.median(apparent_resistivity)
m0 = np.ones(mesh.nC) * np.log(rho0)
# Run inversion
mopt = inv.run(m0)

***Inversion: Show solution***

In [ ]:
def load_leapfrog_geologic_section(filename="./century/geologic_section.csv"):
    """
    Load the geologic cross section. 
    """
    fid = open(filename, 'r')
    lines = fid.readlines()
    data = []
    data_tmp = []
    for line in lines[2:]:
        line_data = (line.split(',')[:3])
        if 'End' in line:
            data.append(np.vstack(data_tmp)[:,[0, 2]])
            data_tmp = []
        else:
            data_tmp.append(np.array(line_data, dtype=float))
    return data

In [ ]:
geologic_section = load_leapfrog_geologic_section()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10, 5))
rho = (mapping*mopt)
norm = LogNorm(vmin=1e1, vmax=1e3)

out = mesh.plot_image(
    rho, pcolor_opts={'norm':norm, 'cmap':'Spectral'}, ax=ax,
)

for data in geologic_section:
    ax.plot(data[:,0], data[:,1], 'k--', alpha=0.5)

ax.set_xlim(core_domain_x)
ax.set_ylim((-400, 0))
cb = plt.colorbar(out[0], fraction=0.05, orientation='horizontal', ax=ax, pad=0.2)
cb.set_label("Resistivity ($\Omega$m)")
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Elevation (m)')
ax.set_aspect(1.5)  # some vertical exxageration
plt.tight_layout()

### Sensitivity analysis

The sensitivity matrix for the DC inverse problem just formulated is defined as
$$
J_{ij} = \frac{\partial d_i}{\partial ln \sigma_j}
$$
where $d_i$ correspond to the ith data functional and $\sigma_j$ is the electrical conductivity in the jth cell of our mesh.

SimPEG simulation classes offer a method called getJ which we will use to have access to the sensitivity matrix. With $J$ in our hands we can explore its content in a number of different ways as illustrated in the sections below.

***Get the sensitivity matrix***

The first step is to get a copy of the sensitivity matrix $J$, what we can acomplish by calling the method getJ().

In [ ]:
simulation_dc._Jmatrix = None # Force computation of new sensitivity matrix
J = simulation_dc.getJ(mopt).copy()

***Average sensitivity***

There are many different ways in which we could look at the information contained in $J$. Let's use the **average sensitivity** $s_j$ as defined in {cite}`https://doi.org/10.1190/segam2016-13842655.1`,
$$
s_j = \frac{1}{nV_j}\sum_{i=1}^n\left|J_{ij}\right|.
$$
It will give us a global qualitative/semi-quantitative view on how the survey layout defined for this inversion problem is sampling the subsurface in the region of interest.

In [ ]:
##
# Define the average sensitivity sj
Vj = mesh.cell_volumes
n, m = J.shape
sj = np.log10(np.divide(np.sum(abs(J), axis=0), n*Vj))

In [ ]:
##
# Plot
fig, ax = plt.subplots(1,1, figsize=(10, 5))
plt.jet()
out = mesh.plot_image(
    sj, 
    ax=ax,
)

for data in geologic_section:
    ax.plot(data[:,0], data[:,1], 'k--', alpha=0.5)

ax.set_xlim(core_domain_x)
ax.set_ylim((-400, 0))
cb = plt.colorbar(out[0], fraction=0.05, orientation='horizontal', ax=ax, pad=0.2)
cb.set_label("Average sensitivity")
ax.set_xlabel('Easting (m)')
ax.set_ylabel('Elevation (m)')
ax.set_aspect(1.5)  # some vertical exxageration
plt.tight_layout()

## Conclusions

In this tutorial we used the resources avaiable in SimPEG for computing and ploting the sensitivity matrix. We used the DC resistivity problem as a motivation example and analysed the sensitivity patterns for a homogeneous half space using two different survey layouts. We also applied this technique for a DC resistivity real data inversion scenario. This kind of analysis can help us decide upon the most appropriate electrode array to use for a particular exploration project. It is also a valuable tool to have in our toolbox for performing the assessment of the solutions obtained after the application of some inversion algorithm as illustrated by the real case example.

## Acknowledgements